In [14]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
import time
import csv

options = Options()
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36")
options.add_argument("--disable-blink-features=AutomationControlled")

service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=options)

driver.execute_cdp_cmd("Page.addScriptToEvaluateOnNewDocument", {
    "source": """
        Object.defineProperty(navigator, 'webdriver', {
            get: () => undefined
        })
    """
})

csvfile = open("see.csv", "w", encoding="utf-8-sig", newline='')
writer = csv.writer(csvfile)
writer.writerow(["الاسم", "السعر"])

url = "https://www.carrefouregypt.com/mafegy/ar/c/best-price"
driver.get(url)
time.sleep(5) 
for _ in range(5):
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(2)

WebDriverWait(driver, 30).until(
    EC.presence_of_all_elements_located((By.CSS_SELECTOR, 'div.css-11qbfb'))
)

products = driver.find_elements(By.CSS_SELECTOR, 'div.css-yqd9tx')

for product in products:
    name_element = product.find_elements(By.CSS_SELECTOR, '[data-testid="product_name"]')
    name = name_element[0].text.strip() if name_element else " غير موجود"

    price_element = product.find_elements(By.CSS_SELECTOR, 'div.css-1b1b7o7')
    price = price_element[0].text.strip() if price_element else " غير موجود"

    writer.writerow([name, price])

csvfile.close()
driver.quit()